# Build song recommendations

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pandas as pd
import numpy as np
import matplotlib as plt
import importlib
from pyspark.ml.feature import Tokenizer, CountVectorizer, MinHashLSH
from pyspark.sql.types import IntegerType, StringType, ArrayType

import mpd

In [ ]:
# Will allow us to embed images in the notebook
%matplotlib inline
# change default plot size
plt.rcParams['figure.figsize'] = (15,10)

## Load and prep data

* Load the full data set
* Vectorize the playlists into sparse vectors
* Extract the vocabulary with tid to allow translation back to track_uri

In [ ]:
mpd_all=mpd.load(spark, "onebig", 1)

## Build track, artist and name features

The track_uri and artist_uri columns are already lists and can be processed in their raw format.

In [ ]:
trackdf = mpd_all.select("pid", "tracks.track_uri")

In [ ]:
artistdf = mpd_all.select("pid", "tracks.artist_uri")

In [ ]:
namedf = mpd.canonicaltokens(mpd_all.select("pid", "name").fillna({"name": ""}), "name", "filtered").drop("name")

In [ ]:
mergeCols = f.udf((lambda x, y: x + y), ArrayType(StringType()))

In [ ]:
featuredf = trackdf.join(artistdf, trackdf.pid == artistdf.pid).drop(artistdf.pid)

In [ ]:
featuredf = featuredf.withColumn("tafeatures", mergeCols("track_uri", "artist_uri")).drop("track_uri").drop("artist_uri")

In [ ]:
featuredf.printSchema()

In [ ]:
featuredf = featuredf.join(namedf, namedf.pid == featuredf.pid).drop(namedf.pid)

In [ ]:
featuredf = featuredf.withColumn("features", mergeCols("filtered", "tafeatures")).drop("tafeatures").drop("filtered")

In [ ]:
featuredf.printSchema()

In [ ]:
featuredf.show(5)

## Build feature vector

In [ ]:
cv = CountVectorizer(inputCol="features", outputCol="featurevector", minDF=2, vocabSize=2000000)

In [ ]:
model=cv.fit(featuredf)

In [ ]:
featurevec = model.transform(featuredf).drop("features")

In [ ]:
featurevec = featurevec.withColumnRenamed("featurevector", "features")

In [ ]:
featurevec.show(5)

## Build LSH 

Eliminate any zero length feature vectors from the input

In [ ]:
vectorlength = f.udf(lambda x: x.numNonzeros(), IntegerType())

In [ ]:
arraylength = f.udf(lambda x: len(x), IntegerType())

In [ ]:
f2 = featurevec.withColumn("vlen", vectorlength(featurevec.features))

In [ ]:
sparsevec = f2.where(f2.vlen > 1)

In [ ]:
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=5)

In [ ]:
mhmodel = mh.fit(sparsevec)

In [ ]:
transform = mhmodel.transform(sparsevec)

In [ ]:
transform.show(5)

In [ ]:
transform.count()

In [ ]:
t2 = transform.withColumn("hlen", arraylength("hashes"))

In [ ]:
t2.orderBy("vlen").show(5)

In [ ]:
t2.orderBy("hlen").show(5)

## Load Challenge set

In [ ]:
mpd_test=spark.read.json("../mpd-challenge/challenge_set.json", multiLine=True)

In [ ]:
cpl=mpd_test.select(f.explode("playlists").alias("playlist"))

In [ ]:
recdf=cpl.select("playlist.name", "playlist.num_holdouts", "playlist.pid", "playlist.num_tracks", "playlist.tracks", "playlist.num_samples")

In [ ]:
chtracks = recdf.select("pid", "tracks.track_uri")

In [ ]:
chartist = recdf.select("pid", "tracks.artist_uri")

In [ ]:
challengedf = chtracks.join(chartist, chtracks.pid == chartist.pid).drop(chartist.pid)

In [ ]:
challengedf = challengedf.withColumn("features", mergeCols(f.col("track_uri"),f.col("artist_uri"))).drop("track_uri").drop("artist_uri")

In [ ]:
tokedf = mpd.canonicaltokens(recdf.select("pid", "name").fillna({"name": ""}), "name", "filtered").drop("name")

In [ ]:
challengedf.printSchema()

In [ ]:
tokedf.printSchema()

In [ ]:
challengedf = challengedf.join(tokedf, tokedf.pid == challengedf.pid).drop(tokedf.pid)

In [ ]:
challengedf.printSchema()

In [ ]:
challengedf = challengedf.withColumn("featurevec", mergeCols("filtered", "features")).drop("filtered").drop("features")

In [ ]:
challengedf = challengedf.withColumnRenamed("featurevec", "features")

In [ ]:
challengedf.printSchema()

In [ ]:
challengedf.show(5)

## Map challenge set into training vocab

In [ ]:
challengevec = model.transform(challengedf).drop("features")

In [ ]:
challengevec = challengevec.withColumnRenamed("featurevector", "features")

In [ ]:
challengevec.printSchema()

In [ ]:
challengevec.show(5)

In [ ]:
c2 = challengevec.withColumn("vlen", vectorlength("features"))

In [ ]:
c2.orderBy("vlen").show(5)

There are challenge set vectors that have a length of zero.  This shouldn't cause any problems because can just recommend top songs from the global data set here.

In [ ]:
c2.where(c2.vlen == 0).describe("pid").show()

## Find playlist matches for one challenge set

Select the first challenge playlist to test.`

In [ ]:
challengevec.cache()

In [ ]:
testpl = challengevec.limit(1)

In [ ]:
testpl.show(truncate=False)

In [ ]:
testvec=challengevec.select("features").rdd.map(lambda x: x.features).take(1)[0]

In [ ]:
testvec

In [ ]:
type(testvec)

In [ ]:
testpid=challengevec.select("pid").rdd.map(lambda x: x.pid).take(1)[0]

In [ ]:
testpid

In [ ]:
mh.getOutputCol()

In [ ]:
mhmodel.params

In [ ]:
transform.printSchema()

In [ ]:
transform.cache()

In [ ]:
hot100 = mhmodel.approxNearestNeighbors(transform, testvec, 100)

In [ ]:
# hot100.explain()

In [ ]:
hot100.count()

In [ ]:
hot100.printSchema()

In [ ]:
hot100.show(5)

In [ ]:
pidlist = hot100.select("pid").toPandas()["pid"].tolist()

In [ ]:
pidlist[0:5]

In [ ]:
type(pidlist)

In [ ]:
recommend = { "pid": testpid, "recpl": [pidlist]}

In [ ]:
recommend

In [ ]:
testpd = pd.DataFrame({"pid":0, "recpl":[]})

In [ ]:
testpd

In [ ]:
testpd.append(pd.DataFrame(recommend))

In [ ]:
recpd = pd.DataFrame(recommend)

In [ ]:
recpd

In [ ]:
recpd.append(pd.DataFrame(recommend))

In [ ]:
spark.createDataFrame(pd.DataFrame(recommend)).show()

## Build nearest neighbor playlists

In [ ]:
def getrecommend(chpl, model, transform):
    #testvec=challenge.select("features").rdd.map(lambda x: x.features).take(1)[0]
    #testpid=challenge.select("pid").rdd.map(lambda x: x.pid).take(1)[0]
    
    testvec = chpl.features
    testpid = chpl.pid
    #print("DEBUG: " + testvec + " " + testpid )
    
    hot100 = model.approxNearestNeighbors(transform, testvec, 100)
    
    pidlist = hot100.select("pid").toPandas()["pid"].tolist()
    
    recommend = { "pid": testpid, "recpl": [pidlist]}
    #print("DEBUG: " + testpid + " " + pidlist)
    
    return recommend
    #return recpd
    #print(requests.pid)
    #testvec = requests.features
    
    #return testpid, testvec

In [ ]:
testpd = pd.DataFrame({"pid":0, "recpl":[]})

In [ ]:
for row in challengevec.limit(100).rdd.collect():
    rec = getrecommend(row, mhmodel, transform)
    testpd = testpd.append(pd.DataFrame(rec))

In [ ]:
testpd